In [1]:
import number_model
import torch

In [2]:
model = number_model.NumberModel(11)  # 0-9, the + and the x symbol
x = torch.randn((1, 3, 640, 480))
output = model(x)
assert output.shape == torch.Size((1, 11)), 'the output shape is wrong'
print(f'In {x.shape} Out {output.shape}')

In torch.Size([1, 3, 640, 480]) Out torch.Size([1, 11])


In [ ]:
# continue